In [2]:
# Cell for imports
import re
import os
import gzip
import time
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from stop_words import get_stop_words
import math


In [4]:
#### Cell of all my function

def list_file_data(nom_directory):
    myListFile = []
    for f in os.listdir(nom_directory):
        if os.path.isdir(f): # si f est un dossier
            os.chdir(f) # On va lister son contenu
            parse()
            os.chdir('../') # On revient au répertoire précédent
        else:
            myListFile.append(f)
        # Traitement sur le fichier f
    return myListFile


def preprocesDataFile(fileName):
    input_file = gzip.open(fileName, 'rb')
    full_text = input_file.read()
    docListNum = re.findall('<doc><docno>(.*?)</docno>(.*?)</doc>', str(full_text).lower().strip())
    list_doc = re.findall('<doc><docno>(.*?)</docno>', str(full_text).strip())
    return docListNum, list_doc

def clean(text1,use_stopword_stemmer):
    
    remove_w = "_:/!?#^*~&()[]{}';$%|,.-"
    stopwords = get_stop_words('english')
    full_text = text1.lower().replace('\\n', '').strip(remove_w).replace("''",' ').replace("'",' ')
    full_text = re.sub(r'[^\w\s]', '', full_text) # Remove all punctuation
    if(use_stopword_stemmer):
        full_text = [stemmer.stem(word) 
                     for word in full_text.split() 
                     if len(word)>1 
                     and word not in stopwords 
                     and not word.isnumeric() 
                     and word.isalnum()] ### remove stops_word and applique stemming
        #full_text = [word for word in full_text.split() if len(word)>1 and word not in stw and not word.isnumeric()] ### remove word and applique stemming
    else:
        full_text = [word 
                     for word in full_text.split() 
                     if len(word)>1  
                     and not word.isnumeric()
                     and word.isalnum()] 
    return ' '.join(full_text)

def countWord(words):   
    word_count = {} # compte l'occurance d'un terme dans tous les documents
    j=0
    for word in words: # On parcours la listes de mots
        j+=1
        word  = word.lower()
        if (len(word)==1) or str(word).isnumeric():
            continue
        if not word in word_count:
            word_count[word] = 1
        else:
            word_count[word] = word_count[word] + 1
    return word_count

def countWordIntoDocs(dico, docno, posting):
    docname = docno
    
    for word, frequence in dico.items():
        posting.setdefault(word,[]).append((docname,frequence)) ### Remplace les lignes de commande suivante:
        """
        if not word in list(posting.keys()):
            posting[word] = [(docname, frequence)]
        else:
            posting[word].append((docname, frequence))
        """
    return posting

def doc_len(list_terms):
    dl = [(doc_n, len(len_doc)) for doc_n,len_doc in list_terms.items()]
    return dl

### Function to check after
def document_lenght(list_terms,posting_list):
    dl = {}
    for doc, value in list_terms.items():
        somme_tf = 0
        for term in value:
            if term.isnumeric():
                continue
            for val in sorted(posting_list[term]):
                if val[0]==doc:
                    somme_tf+=val[1]
        dl[doc]=somme_tf
        return dl
    
def vocabulary_size(posting_list):
    return len(posting_list.keys())


def term_len(posting_list):
    # Return 
    return [(term, len(term)) for term in posting_list.keys()]


def collection_term_freq(posting_list):
    c_size={}
    dl = {}
    for term,values in posting_list.items(): # get the term
        somme=0
        for v in values:
            somme+=v[1]
        dl[term]=[(somme,len(values))] #Frequence d'apparution du doc et nombre de doc dans lequel il apparait
    return dl

## Pour avoir une courbe avec le plot
def plot_datas(data, title, label_x, label_y):
    x = list()
    y = list()
    for key, values in data.items():
        x.append(key)
        y.append(values[0])
    plt.plot(x,y, color='blue',marker='o',linestyle='solid')
    #Titre
    plt.title(title)
    # label
    plt.xlabel(label_x)
    plt.ylabel(label_y)
    plt.show()
    
## Fonction de traintement du texte   
def text_mining(fileName,use_stopword_stemmer=bool()):
    start = time.time()
    docListNum, list_doc = preprocesDataFile(fileName)
    posting_list = {}
    file_number = fileName.split('/')[1].split('-',1)
    dl = list()
    for i in range(len(list_doc)):
        text_clean = clean(docListNum[i][1],use_stopword_stemmer)
        list_terms[list_doc[i]] = text_clean.split() # Here we create a dictionary of Here we create a dictionary of 
                                                                              # each docments with its terms
        lt = text_clean.split()
        current_dico = countWord(lt)
        posting_list = countWordIntoDocs(current_dico, list_doc[i], posting_list)
    end = time.time()
    elapsed = end - start
    tf = term_len(posting_list)
    file_indexing_infos=(round(elapsed,3),tf)
    return posting_list, list_terms



def get_statistics(posting_list,list_terms):
    stat = {}
    # document length
    stat['df'] = doc_len(list_terms)
    # Number of a doc
    stat['n_doc'] = len(stat['df'])
    # term length
    stat['tl'] = term_len(posting_list)
    #vocabulary size
    stat['voc_size'] = vocabulary_size(posting_list)
    #collection frequency of terms
    stat['colec_freq'] = collection_term_freq(posting_list)
    return stat

In [5]:
### Cell to initialise variable.
directory = "Practice_02_data/"
list_data = list_file_data(directory)
list_data = sorted(list_data)

posting_list = {}
posting_list_global = {}
file_indexing_infos = {}
dl = {}
tf = {}
list_char = "_:/!?#^*~&()[]{}';$%|,.-"
stopwords = get_stop_words('english')
list_terms = {}
stemmer = PorterStemmer()

In [14]:
default_filename = str(directory+'01-Text_Only-Ascii-Coll-1-10-NoSem.gz')
pl,lt = text_mining(default_filename,True)
stat = get_statistics(pl,lt)

[('1000', 11), ('3000', 14)]

In [13]:
query = "web ranking scoring algorithm"
query_clean = clean(query, True)
c_tf = collection_term_freq(pl)

 
def scoring_doc(query_clean, pl, lt):
    rsv = {}
    stat = get_statistics(pl,lt)
    for q in query_clean.split():
        if q in pl.keys():
            #tf = [f[1] for f in pl[q]]
            for v in pl[q]:
                tf = (v[1]/len(lt[v[0]]))
                idf = math.log(stat['n_doc'] / len(pl[q]))
                #val = (1+math.log(value[1]))/math.log(stat['n_doc']/tf)
                rsv.setdefault(v[0],[]).append((tf*idf))
    rsv = [(doc,sum(v)) for doc,v in rsv.items()]
    return rsv

rsv = scoring_doc(query_clean, pl, lt)
rsv

[('1000', 0.0011535997459890008)]

In [8]:
def smart_ltn(posting_list,n, c_tf):
    #=(1+LOG(tf))*LOG(n/c_tl)
    ltn = {}
    for key, value in posting_list.items():
        for v in value:
            tf = 1+math.log(v[1])
        for v in c_tf[key]:
            c_tl = v[1]
        if c_tl==0:
            ltn[key] = 0
        else:
            ltn[key] = tf*math.log(n/c_tl)
    ltn_sum = sum(ltn.values())
    return ltn_sum

In [9]:
rsv = {} 
for q in query_clean.split():
    if q in pl.keys():
        tf = sum([f[1] for f in pl[q]])
        for value in pl[q]:
            val = (1+math.log(value[1]))/math.log(stat['n_doc']/tf)
            rsv.setdefault(value[0],[]).append((val))
rsv = [(doc,sum(v)) for doc,v in rsv.items()]
rsv


[('1000', 1.052011492633005)]

10

In [32]:
def smart_ltn_2(posting_list,list_terms):
    stat = get_statistics(posting_list,list_terms)
    n = stat['n_doc']
    df = 0
    #(1+LOG(tf))*LOG(n/df)
    #tf is the frequence of term on a document
    #df number of document where the term appear
    ltn_value = {} # key doc number, value (terme, ltn)
    for term, value in posting_list.items():
        df =len(value) #
        for v in value:
            tf = v[1]
            if df == 0:
                ltn=0
            else:
                ltn = (1+math.log(tf))*math.log(n/df)
            ltn_value.setdefault(v[0],[]).append((term,ltn))
    return ltn_value

smart_ltn_2(pl,lt)

{'1000': [('hercul', 10.617030850673812),
  ('poirot100024368565120081007t164526zlightmouse4469495hercul',
   2.302585092994046),
  ('poirot', 14.259810126415148),
  ('charactersfict', 2.302585092994046),
  ('privat', 4.956169966304042),
  ('investigatorshercul', 2.302585092994046),
  ('poirotseri', 2.302585092994046),
  ('booksinfobox', 2.302585092994046),
  ('characterpriv', 2.302585092994046),
  ('dectect', 2.302585092994046),
  ('former', 2.725015263724081),
  ('retir', 8.95791201593675),
  ('detect', 7.376889151910965),
  ('polic', 5.468701474522307),
  ('officercurtain', 2.302585092994046),
  ('david', 6.428263737069489),
  ('suchetpet', 2.302585092994046),
  ('ustinovalbert', 2.302585092994046),
  ('finneyse', 2.302585092994046),
  ('belowpolic', 2.302585092994046),
  ('officeragatha', 2.302585092994046),
  ('christieroman', 2.302585092994046),
  ('catholicdedee2belgianhercul', 2.302585092994046),
  ('poirotmaleth', 2.302585092994046),
  ('mysteri', 9.52232733376156),
  ('affair

    # document length
    stat['df'] = doc_len(list_terms)
    # Number of a doc
    stat['n_doc'] = len(stat['df'])
    # term length
    stat['tl'] = term_len(posting_list)
    #vocabulary size
    stat['voc_size'] = vocabulary_size(posting_list)
    #collection frequency of terms
    stat['colec_freq'] = collection_term_freq(posting_list)
    